In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from selenium import webdriver

### importing the list of all formula one races from wikipedia

In [34]:
url = "https://en.wikipedia.org/wiki/List_of_Formula_One_circuits"
page = requests.get(url)

In [35]:
soup = BeautifulSoup(page.text, "html")
# print(soup.prettify())

In [36]:
tables = soup.find_all("table")
# print(tables)

In [37]:
print(len(tables))

4


In [38]:
for table in tables:
    print(f"Table: <{table.name}>")
    print(f"ID: {table.get('id')}")
    print(f"Class: {table.get('class')}")

Table: <table>
ID: None
Class: ['sidebar', 'sidebar-collapse', 'nomobile', 'nowraplinks', 'plainlist']
Table: <table>
ID: None
Class: ['wikitable']
Table: <table>
ID: None
Class: ['wikitable', 'sortable']
Table: <table>
ID: None
Class: ['nowraplinks', 'mw-collapsible', 'autocollapse', 'navbox-inner']


In [39]:
# print(tables[2])

we got the table we were looking for

In [40]:
circuts = tables[2]
columns = circuts.find_all("th")
column_names = [title.text.strip() for title in columns]
print(column_names)

['Circuit', 'Map', 'Type', 'Direction', 'Location', 'Country', 'Last length used', 'Turns', 'Grands Prix', 'Season(s)', 'Grands Prix held']


we won't use "Map" column as it is an Image and it's useless for our purpose

In [9]:
column_names = [name for name in column_names if name != "Map"]

we will also extract Latitude and Longitude

In [41]:
column_names.extend(["Latitude", "Longitude"])

In [42]:
print(column_names)

['Circuit', 'Map', 'Type', 'Direction', 'Location', 'Country', 'Last length used', 'Turns', 'Grands Prix', 'Season(s)', 'Grands Prix held', 'Latitude', 'Longitude']


we have the column names, let's move on to the data and put it in a dataframe

In [11]:
df = pd.DataFrame(columns = column_names)
df

,Circuit,Type,Direction,Location,Country,Last length used,Turns,Grands Prix,Season(s),Grands Prix held,Coordinates


In [32]:
# function for extracting coordinates for each circut from the link of an external corresponding circut page

def extract_coordinates(current_row):
    link_tag = current_row[0].find("a")
    if link_tag and "href" in link_tag.attrs:
        link = link_tag["href"]
        full_link = "https://en.wikipedia.org" + link 
        # print(full_link)
    else:
        print("error")
        
    driver = webdriver.Chrome()
    driver.get(full_link)
    # time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table", class_="infobox")

    coordinates_row = table.find("th", string=lambda text: text and "Coordinates" in text)
    
    td = coordinates_row.find_next_sibling("td")
    latitude = td.find("span", class_="latitude")
    longitude = td.find("span", class_="longitude")
        
    driver.quit()

    if latitude and longitude:
        return [latitude.text, longitude.text]
    else:
        return [0,0]
    
    

In [31]:
data = circuts.find_all("tr")
for row in data[1:]:
    current_row = row.find_all("td")
    extract_coordinates(current_row)
    current_row_data = [current_row[k].text.strip() for k in range(len(current_row)) if k != 1] # omit image data from Map column
    # df.loc[len(df)] = current_row_data

Latitude: 34°55′50″S
Longitude: 138°37′14″E
Latitude: 33°34′43″N
Longitude: 7°41′15″W
Latitude: 53°28′37″N
Longitude: 2°56′26″W
Latitude: 37°50′59″S
Longitude: 144°58′6″E
Latitude: 37°13′55″N
Longitude: 8°37′55″W
Latitude: 38°45′3″N
Longitude: 9°23′39″W


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.69)
Stacktrace:
0   chromedriver                        0x000000010333e654 cxxbridge1$str$ptr + 2723108
1   chromedriver                        0x00000001033368c8 cxxbridge1$str$ptr + 2690968
2   chromedriver                        0x0000000102e8a714 cxxbridge1$string$len + 90428
3   chromedriver                        0x0000000102e64740 chromedriver + 132928
4   chromedriver                        0x0000000102ef99e4 cxxbridge1$string$len + 545804
5   chromedriver                        0x0000000102f12908 cxxbridge1$string$len + 647984
6   chromedriver                        0x0000000102ec59c8 cxxbridge1$string$len + 332784
7   chromedriver                        0x0000000103302278 cxxbridge1$str$ptr + 2476360
8   chromedriver                        0x000000010330550c cxxbridge1$str$ptr + 2489308
9   chromedriver                        0

In [51]:
df

,Circuit,Type,Direction,Location,Country,Last length used,Turns,Grands Prix,Season(s),Grands Prix held
0,Adelaide Street Circuit,Street circuit,Clockwise,Adelaide,Australia,3.780 km (2.349 mi),16,Australian Grand Prix,1985–1995,11
1,Ain-Diab Circuit,Road circuit,Clockwise,Casablanca,Morocco,7.618 km (4.734 mi),18,Moroccan Grand Prix,1958,1
2,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828 km (3.000 mi),12,British Grand Prix,"1955, 1957, 1959, 1961–1962",5
3,Albert Park Circuit *,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi),16,Australian Grand Prix,"1996–2019, 2022–2025",28
4,Algarve International Circuit,Race circuit,Clockwise,Portimão,Portugal,4.653 km (2.891 mi),15,Portuguese Grand Prix,2020–2021,2
...,...,...,...,...,...,...,...,...,...,...
72,TI Circuit Aida,Race circuit,Clockwise,Mimasaka,Japan,3.703 km (2.301 mi),11,Pacific Grand Prix,1994–1995,2
73,Valencia Street Circuit,Street circuit,Clockwise,Valencia,Spain,5.419 km (3.367 mi),25,European Grand Prix,2008–2012,5
74,Watkins Glen International,Race circuit,Clockwise,Watkins Glen,United States,5.430 km (3.374 mi),10,United States Grand Prix,1961–1980,20
75,Yas Marina Circuit *,Race circuit,Anti-clockwise,Abu Dhabi,United Arab Emirates,5.281 km (3.281 mi),15,Abu Dhabi Grand Prix,2009–2024,16


won't need this column as well

df = df.drop(columns=['Grands Prix held'])

In [52]:
df.to_csv("circuts.csv", index=False)

## transform the table to SCD2 type, by dividing 'Season(s)' column to: year_from, year_to, isActive 

In [6]:
df = pd.read_csv('circuts.csv')

In [7]:
def parse_seasons(seasons_str, isActive): # is active from the table as the seasons date's not always match
    periods = []
    parts = [s.strip() for s in seasons_str.split(',')]
    for part in parts:
        if ']' in part:
            part = part[0:-3]
            print(part)
        if '–' in part:
            start, end = part.split('–')
            if end == "2025":
                periods.append((int(start), int(end), True))
            elif end == "2024" and isActive:
                periods.append((int(start), 2025, True))
            else:
                periods.append((int(start), int(end), False))
        else:
            if part == "2025":
                periods.append((int(part), int(part), True))
            elif part == "2024" and isActive:
                periods.append((2025, 2025, True))
            else:
                periods.append((int(part), int(part), False))
    return periods

In [8]:
df.head(20)

,Circuit,Type,Direction,Location,Country,Last length used,Turns,Grands Prix,Season(s),Grands Prix held
0,Adelaide Street Circuit,Street circuit,Clockwise,Adelaide,Australia,3.780 km (2.349 mi),16,Australian Grand Prix,1985–1995,11
1,Ain-Diab Circuit,Road circuit,Clockwise,Casablanca,Morocco,7.618 km (4.734 mi),18,Moroccan Grand Prix,1958,1
2,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828 km (3.000 mi),12,British Grand Prix,"1955, 1957, 1959, 1961–1962",5
3,Albert Park Circuit *,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi),16,Australian Grand Prix,"1996–2019, 2022–2025",28
4,Algarve International Circuit,Race circuit,Clockwise,Portimão,Portugal,4.653 km (2.891 mi),15,Portuguese Grand Prix,2020–2021,2
5,Autódromo do Estoril,Race circuit,Clockwise,Estoril,Portugal,4.360 km (2.709 mi),13,Portuguese Grand Prix,1984–1996,13
6,Autódromo Hermanos Rodríguez *,Race circuit,Clockwise,Mexico City,Mexico,4.304 km (2.674 mi),17,"Mexican Grand Prix,Mexico City Grand Prix","1963–1970, 1986–1992, 2015–2019, 2021–2024",24
7,Autódromo Internacional do Rio de Janeiro,Race circuit,Anti-clockwise,Rio de Janeiro,Brazil,5.031 km (3.126 mi),11,Brazilian Grand Prix,"1978, 1981–1989",10
8,Autodromo Internazionale del Mugello,Race circuit,Clockwise,Scarperia e San Piero,Italy,5.245 km (3.259 mi),14,Tuscan Grand Prix,2020,1
9,Autodromo Internazionale Enzo e Dino Ferrari *,Race circuit,Anti-clockwise,Imola,Italy,4.909 km (3.050 mi),17,"Italian Grand Prix,San Marino Grand Prix,Emili...","1980–2006, 2020–2022, 2024",31


In [9]:
print(df.loc[3, "Season(s)"])
print(parse_seasons(df.loc[3, "Season(s)"]))

1996–2019, 2022–2025


TypeError: parse_seasons() missing 1 required positional argument: 'isActive'

In [10]:
def build_new_rows(row):
    periods_str = row['Season(s)']
    isActive = True if '*' in row['Circuit'] else False
    result = []
    periods = parse_seasons(periods_str, isActive)

    # sort by start year
    periods.sort()
    
    result = []
    
    for start, end, active in periods:
        new_row = row.to_dict()
        new_row['from'] = start
        new_row['to'] = end
        new_row['isActivePeriod'] = active
        result.append(new_row)
    return result

In [11]:
transformed_df = []
for _, row in df.iterrows():
    transformed_df.extend(build_new_rows(row))
    print(len(transformed_df))

scd2_df = pd.DataFrame(transformed_df)

1
2
6
8
9
10
14
16
17
20
24
26
31
32
2012–2025
34
36
46
47
48
49
52
53
54
57
58
60
66
74
79
83
85
87
96
101
104
106
107
110
116
117
118
119
121
130
131
133
135
136
137
140
141
142
144
146
147
151
155
157
167
168
169
171
2014–2024
174
175
180
181
182
183
185
1987–2024
202
203
206
207
208
209
210
211


In [12]:
scd2_df.head(15)

,Circuit,Type,Direction,Location,Country,Last length used,Turns,Grands Prix,Season(s),Grands Prix held,from,to,isActivePeriod
0,Adelaide Street Circuit,Street circuit,Clockwise,Adelaide,Australia,3.780 km (2.349 mi),16,Australian Grand Prix,1985–1995,11,1985,1995,False
1,Ain-Diab Circuit,Road circuit,Clockwise,Casablanca,Morocco,7.618 km (4.734 mi),18,Moroccan Grand Prix,1958,1,1958,1958,False
2,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828 km (3.000 mi),12,British Grand Prix,"1955, 1957, 1959, 1961–1962",5,1955,1955,False
3,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828 km (3.000 mi),12,British Grand Prix,"1955, 1957, 1959, 1961–1962",5,1957,1957,False
4,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828 km (3.000 mi),12,British Grand Prix,"1955, 1957, 1959, 1961–1962",5,1959,1959,False
5,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828 km (3.000 mi),12,British Grand Prix,"1955, 1957, 1959, 1961–1962",5,1961,1962,False
6,Albert Park Circuit *,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi),16,Australian Grand Prix,"1996–2019, 2022–2025",28,1996,2019,False
7,Albert Park Circuit *,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi),16,Australian Grand Prix,"1996–2019, 2022–2025",28,2022,2025,True
8,Algarve International Circuit,Race circuit,Clockwise,Portimão,Portugal,4.653 km (2.891 mi),15,Portuguese Grand Prix,2020–2021,2,2020,2021,False
9,Autódromo do Estoril,Race circuit,Clockwise,Estoril,Portugal,4.360 km (2.709 mi),13,Portuguese Grand Prix,1984–1996,13,1984,1996,False


In [14]:
scd2_df['from'] = scd2_df['from'].astype(int)
scd2_df['to'] = scd2_df['to'].astype(int)

In [16]:
scd2_df[(scd2_df['from'] <= 2014) & (scd2_df['to'] >= 2014)]

,Circuit,Type,Direction,Location,Country,Last length used,Turns,Grands Prix,Season(s),Grands Prix held,from,to,isActivePeriod
6,Albert Park Circuit *,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi),16,Australian Grand Prix,"1996–2019, 2022–2025",28,1996,2019,False
22,Autodromo José Carlos Pace *,Race circuit,Anti-clockwise,São Paulo,Brazil,4.309 km (2.677 mi),15,"Brazilian Grand Prix,São Paulo Grand Prix","1973–1977, 1979–1980, 1990–2019, 2021–2024",41,1990,2019,False
25,Autodromo Nazionale di Monza *,Race circuit,Clockwise,Monza,Italy,5.793 km (3.600 mi),11,Italian Grand Prix,"1950–1979, 1981–2024",74,1981,2025,True
33,Bahrain International Circuit *,Race circuit,Clockwise,Sakhir,Bahrain,5.412 km (3.363 mi),15,"Bahrain Grand Prix,Sakhir Grand Prix","2004–2010, 2012–2025[a]",22,2012,2025,True
53,Circuit de Barcelona-Catalunya *,Race circuit,Clockwise,Montmeló,Spain,4.657 km (2.894 mi),14,Spanish Grand Prix,1991–2024,34,1991,2025,True
55,Circuit de Monaco *,Street circuit,Clockwise,Monte Carlo,Monaco,3.337 km (2.074 mi),19,Monaco Grand Prix,"1950, 1955–2019, 2021–2024",70,1955,2019,False
73,Circuit de Spa-Francorchamps *,Race circuit,Clockwise,Stavelot,Belgium,7.004 km (4.352 mi),20,Belgian Grand Prix,"1950–1956, 1958, 1960–1968, 1970, 1983, 1985–2...",57,2007,2025,True
81,Circuit Gilles-Villeneuve *,Street circuit,Clockwise,Montreal,Canada,4.361 km (2.710 mi),13,Canadian Grand Prix,"1978–1986, 1988–2008, 2010–2019, 2022–2024",43,2010,2019,False
85,Circuit of the Americas *,Race circuit,Anti-clockwise,Austin,United States,5.513 km (3.426 mi),20,United States Grand Prix,"2012–2019, 2021–2024",12,2012,2019,False
127,Hockenheimring,Race circuit,Clockwise,Hockenheim,Germany,4.574 km (2.842 mi),16,German Grand Prix,"1970, 1977–1984, 1986–2006, 2008, 2010, 2012, ...",37,2014,2014,False
